In [1]:
import ee
import geemap
import os


In [2]:
# geemap.update_package()

In [3]:
Map = geemap.Map()


In [4]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [5]:
class_palette = ['0c40f1', #1 agua*
 'f0ff00', #2 cultivo*
 'f99504', #3 estepa*
 '17d4c6', #4 humedal*
 '24ff00', #5 matorral abierto*
 '2de50f', #6 matorral arborescente*
 'ccba7d', #7 plantaciones*
 'dec61a', #8 praderas*
 '28d909', #9 primario*
 '4e8c44', #10 secundario*
 '685205', #11 suelo desnudo*

]

In [6]:
class_values = [1,2,3,4,5,6,7,8,9,10,11
    ]

In [7]:
landcover_2018.bandNames().getInfo()

['b1']

In [8]:
clasificacion_2018.bandTypes().getInfo()


{'b1': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}}

In [9]:
# land2018 = landcover_2018.set('classification_class_values', class_values)
# land2018f = land2018.set('classification_class_palette', class_palette)

In [10]:
# class2018 = clasificacion_2018.set('classification_class_values', class_values)
# class2018f = class2018.set('classification_class_palette', class_palette)

In [11]:
# Map.addLayer(land2018f, {}, 'landcover_prov_coyhaique' )
# Map.addLayer(class2018f,{},'clasificacion 2018')

In [12]:
mosaico.bandNames().getInfo()

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

In [13]:
work_dir = os.path.expanduser('../Data/shapes/')

merged_shp = os.path.join(work_dir, 'roi_h.shp')
val_shp= os.path.join(work_dir, 'puntos_validacion.shp')

In [14]:
merged = geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [15]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')

In [16]:
dataset = mosaico.sampleRegions(**{
    'collection': merged,
    'properties': ['IDH'],
    'scale' : 30 ,
})

In [17]:
datasetval = mosaico.sampleRegions(**{
    'collection': val,
    'properties': ['IDH'],
    'scale': 30,
})

In [18]:
print(dataset.size().getInfo())

14099


In [19]:
print(datasetval.size().getInfo())

314


In [20]:
dataset.size().getInfo()

14099

In [21]:
print(dataset.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'IDH': 9, 'b1': 0.026168961077928543, 'b2': 0.04253531992435455, 'b3': 0.03470490872859955, 'b4': 0.33766427636146545, 'b5': 0.1495322287082672, 'b6': 0.06873124837875366, 'b7': 0.8135994672775269, 'b8': 0.7762474417686462, 'b9': 0.5209251642227173}}


In [22]:
# Guardamos las bandas
bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

# Esta propiedad guarda las clases de landcover
label = 'IDH'

# Overlay the points on the imagery to get training.
# sample = mosaico.select(bands).sampleRegions(**{
#   'collection': dataset,
#   'properties': ['RF'],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
# sample = dataset.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [23]:
# print(training.first().getInfo())

In [24]:
# print(validation.first().getInfo())

In [25]:
classifier = ee.Classifier.smileRandomForest(10).train(dataset, label, bands)

In [26]:
# Clasificando con las mismas bandas de entrenamiento
result = mosaico.select(bands).classify(classifier)

# # Display the clusters with random colors.
# Map.addLayer(result.randomVisualizer(), {}, 'classificado')
# Map

In [27]:
result.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'dimensions': [6083, 9750],
   'crs': 'EPSG:32719',
   'crs_transform': [30, 0, 179085, 0, -30, 5153095]}],
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-70.71978119886894, -43.7621302946721],
    [-70.71984230426219, -43.762120959847216],
    [-72.98275024455971, -43.70561719895318],
    [-72.98292936081644, -43.70572079387638],
    [-72.98313712829581, -43.705812701067146],
    [-73.02744506941205, -44.362573403829224],
    [-73.07328396187123, -45.01916085040947],
    [-73.12072282029395, -45.67563442548405],
    [-73.16983051350653, -46.331992631080205],
    [-73.16968129990005, -46.33212249547371],
    [-73.1695568677538, -46.33226430394552],
    [-73.16949325899127, -46.33227507995767],
    [-70.80072781794739, -46.3941742696168],
    [-70.80054376625334, -46.39406743407541],
    [-

In [28]:
class_values = [1,2,3,4,5,6,7,8,9,10,11
    ]

In [29]:
land = result.set('classification_class_values', class_values)
landcoverf = land.set('classification_class_palette', class_palette)

In [30]:
Map.addLayer(landcoverf, {}, 'Land cover Random Forest')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [31]:
train_accuracy = classifier.confusionMatrix()

In [32]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3237, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 806, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 620, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 2, 0, 0, 133, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 222, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 158, 0, 0, 2, 2, 0],
 [0, 0, 0, 0, 0, 0, 0, 3098, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0, 254, 0, 0, 2],
 [0, 0, 0, 0, 0, 1, 4, 0, 0, 1498, 3, 0],
 [0, 0, 0, 0, 0, 0, 3, 0, 0, 9, 1599, 0],
 [0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 4, 2435]]

In [33]:
train_accuracy.accuracy().getInfo()

0.9972338463720831

In [34]:
train_accuracy.kappa().getInfo()

0.9967011447389702

In [35]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [1],
 [0.998389694041868],
 [0.9851851851851852],
 [1],
 [0.9753086419753086],
 [1],
 [0.9844961240310077],
 [0.9946879150066401],
 [0.9925512104283054],
 [0.9967253376995497]]

In [36]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9993825254708243,
  0.9975247524752475,
  0.9951845906902087,
  1,
  0.9955156950672646,
  0.9575757575757575,
  1,
  0.996078431372549,
  0.9927104042412194,
  0.9944029850746269,
  0.9987694831829368]]

In [37]:
validated = datasetval.classify(classifier)

In [38]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'IDH': 1,
  'b1': 0.01979999989271164,
  'b2': 0.017999999225139618,
  'b3': 0.010499999858438969,
  'b4': 0.01730000041425228,
  'b5': 0.00989999994635582,
  'b6': 0.0071000000461936,
  'b7': 0.24460433423519135,
  'b8': -0.019829994067549706,
  'b9': 0.019325504079461098,
  'classification': 1}}

In [39]:
test_accuracy = validated.errorMatrix('IDH', 'classification')

In [40]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 22, 0, 0, 0, 0, 0, 0, 0, 5],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 2, 0, 0, 2, 6, 0, 1, 3, 0, 3],
 [0, 0, 3, 0, 0, 0, 7, 2, 0, 15, 3, 0],
 [0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0],
 [0, 0, 14, 1, 0, 1, 2, 0, 6, 1, 0, 7],
 [0, 0, 5, 0, 0, 0, 3, 2, 0, 97, 13, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 1, 0, 0, 2, 0, 0, 68]]

In [41]:
test_accuracy.accuracy().getInfo()

0.678343949044586

In [42]:
test_accuracy.kappa().getInfo()

0.5851939049113857

In [43]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.8],
 [1],
 [0.8148148148148148],
 [0],
 [0.11764705882352941],
 [0.23333333333333334],
 [0.75],
 [0.1875],
 [0.8016528925619835],
 [0],
 [0.9315068493150684]]

In [44]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  1,
  0.14285714285714285,
  0.88,
  0,
  0.5,
  0.3888888888888889,
  0.42857142857142855,
  0.6666666666666666,
  0.8220338983050848,
  0,
  0.8095238095238095]]